# CS429: Information Retrieval

<br>

## Lecture 16: Naive Bayes

<br>

### Dr. Aron Culotta
### Illinois Institute of Technology

Recall classification problem notation:


- $\vec{x} \in \mathcal{X}$ &nbsp;&nbsp;&nbsp;&nbsp; *instance*, *example*, *input*
  - e.g., an email
- $y \in \mathcal{Y}$ &nbsp;&nbsp;&nbsp;&nbsp; *target*, *class*, *label*, *output*
  - e.g., $y=1$: spam ; $y=-1$: not spam
- $f: \mathcal{X} \mapsto \mathcal{Y}$ &nbsp;&nbsp;&nbsp;&nbsp; *hypothesis*, *learner*, *model*, *classifier*
  - e.g., if $x$ contain the word *free*, $y$ is $1$.

** Training data:**

We are given training data $D = \{(\vec{x}_1, y_1), \ldots, (\vec{x}_n, y_n)\}$

||free|money| |*label*|
|:--:|:--------:|:--------:|:--:|:--:|
||$x_{i1}$|$x_{i2}$| | $y_i$ |
|$x_1$|0|0||-1| 
|$x_2$|1|0|| 1|
|$x_3$|1|1||-1|
|$x_4$|1|0||-1|
|$x_5$|1|1||1|
|$x_6$|0|0||1|
|$x_7$|0|1||-1|

How to classify a new instance?  
 "free money" -> $\{1,1\}$


**Recall Bayes' Rule:**

$$
p(y|x) = \frac{p(x|y)p(y)}{p(x)}
$$

To classify an instance $\vec{x}$, we need $p(y|\vec{x})$.

E.g., if $p(y=1|\vec{x}) > p(y=-1|\vec{x})$, then classify $\vec{x}$ as 1.

Using Bayes' rule, we can rewrite $p(y|\vec{x})$ as:

$$
p(y|\vec{x}) = \frac{p(\vec{x}|y)p(y)}{p(\vec{x})}
$$

<br><br><br>
Three terms:

- $p(y)$: **prior** probability of class y.
$$p(y=1) = \frac{\sum_{(x_i, y_i) \in D} 1[y_i=1]}{|D|}$$
 - $1[x]= 1 $ if $x$ is True, $0$ otherwise.
<br><br><br>

- $p(\vec{x})$: **evidence** (probability of this document)
$$p(\vec{x}) = \sum_{y^k \in \{-1, 1\}} p(\vec{x} | y=y^k)p(y=y^k)$$
  - This is just the sum of the numerator for all settings of $y$
  
<br><br><br>

- $p(\vec{x}|y)$ **likelihood**
  - This is harder to estimate.
 

<br><br><br><br>

**Estimating $p(\vec{x}|y)$:**

<table width=40%>
<tr> <th width=10%> free</th> <th width=10%>money</th> <th width=40%>$p(\vec{x}|y=1)$</th> <th width=40%>$p(\vec{x}|y=-1)$</th> </tr>
<tr><td>0</td><td>0</td><td>?</td><td>?</td></tr>
<tr><td>0</td><td>1</td><td>?</td><td>?</td></tr>
<tr><td>1</td><td>0</td><td>?</td><td>?</td></tr>
<tr><td>1</td><td>1</td><td>?</td><td>?</td></tr>
</table>
(assuming binary feature values)


We'd like to be able to compute this just like $p(y)$:

$$
p(\vec{x}=\{0,0\}|y=1) = \frac{\sum_{(x_i, y_i) \in D} 1[\vec{x}_i=\{0,0\}]}{\sum_i 1[y_i=1]}
$$
i.e., what percentage of documents with label $y=1$ have feature vector $\{0, 0\}$?

<br><br><br>
Clearly, this does not scale. 

Size of table above?
<br><br><br>

$k*2^d$, where $d$ is the number of features and $k$ is the number of classes. In text classification, $d$ is often in the tens of thousands.

Even more ridiculous if we move from binary features to feature counts.

Instead, we make a conditional independence assumption (just like in the [Binary Independence Model](https://github.com/iit-cs429/main/tree/master/lectures/lec11)).

This assumption is what is Naïve about Naïve Bayes.

Racell, conditional independence means that $p(a,b|c) = p(a|c)p(b|c)$. Here, we assume each feature value is independent of others given the class label: 

$$p(\vec{x_i} | y=1) =  p(\{x_{i1}, x_{i2}, \ldots x_{id}) \approx  p(\{x_{i1}|y=1) p(x_{i2}|y=1) \ldots p(x_{id}|y=1)$$

Thus, our table of parameters to estimate becomes linear:

<table width=40%>
<tr> <th width=10%> free</th>  <th width=40%>$p(free|y=1)$</th> <th width=40%>$p(free|y=-1)$</th> </tr>
<tr><td>0</td><td>?</td><td>?</td></tr>
<tr><td>1</td><td>?</td><td>?</td></tr>
</table>


<table width=40%>
<tr> <th width=10%> money</th>  <th width=40%>$p(money|y=1)$</th> <th width=40%>$p(money|y=-1)$</th> </tr>
<tr><td>0</td><td>?</td><td>?</td></tr>
<tr><td>1</td><td>?</td><td>?</td></tr>
</table>

Number of parameters: $k*d$, where $d$ is the number of features and $k$ is the number of classes.

<br><br><br>

Our classification formula then becomes:

$$
p(y|\vec{x}) = \frac{p(y)\prod_j p(x_{ij}|y)}{p(\vec{x})}
$$



**Estimating $p(x_{ij}|y)$:**

When term features are **binary**, we call this a **[Bernoulli](https://en.wikipedia.org/wiki/Bernoulli_distribution)** model (word presence is outcome of a biased coin flip)

$$p(x_{k}=1|y=1) = \frac{\sum_{(x_i, y_i) \in D}1[x_{ik}=1 \wedge y_i=1]}{\sum_{(x_i, y_i) \in D} 1[y_i=1]}$$
i.e., what proportion of documents where $y=1$ have term $k$?

<br><br>

We can compute similar values for term absence, as well as for other classes:

$$p(x_{k}=0|y=1) = \frac{\sum_{(x_i, y_i) \in D}1[x_{ik}=0 \wedge y_i=1]}{\sum_{(x_i, y_i) \in D} 1[y_i=1]}$$
i.e., what proportion of documents where $y=1$ **do not** have term $k$?

<br><br>

$$p(x_{k}=1|y=-1) = \frac{\sum_{(x_i, y_i) \in D}1[x_{ik}=1 \wedge y_i=-1]}{\sum_{(x_i, y_i) \in D} 1[y_i=-1]}$$
i.e., what proportion of documents where $y=-1$ have term $k$?

<br><br>

$$p(x_{k}=0|y=-1) = \frac{\sum_{(x_i, y_i) \in D}1[x_{ik}=0 \wedge y_i=-1]}{\sum_{(x_i, y_i) \in D} 1[y_i=-1]}$$
i.e., what proportion of documents where $y=-1$ **do not** have term $k$?



Given a specific set of training data:

||free|money| |*label*|
|:--:|:--------:|:--------:|:--:|:--:|
||$x_{i1}$|$x_{i2}$| | $y_i$ |
|$x_1$|0|0||-1| 
|$x_2$|1|0|| 1|
|$x_3$|1|1||-1|
|$x_4$|1|0||-1|
|$x_5$|1|1||1|
|$x_6$|0|0||1|
|$x_7$|0|1||-1|


- $p(free=1|y=1) = \frac{2}{3}$
- $p(free=0|y=1) = \frac{1}{3}$
- $p(free=1|y=-1) = \frac{2}{4}$
- $p(free=0|y=-1) = \frac{2}{4}$
<br><br>
- $p(money=1|y=1) = \frac{1}{3}$
- $p(money=0|y=1) = \frac{2}{3}$
- $p(money=1|y=-1) = \frac{2}{4}$
- $p(money=0|y=-1) = \frac{2}{4}$

<br>

Note that $p(x=0|y=1) = 1 - p(x=1|y=1)$

<br><br>

**Estimate Prior:**

- $p(y=1) = \frac{3}{7}$
- $p(y=-1) = \frac{4}{7}$

<br><br>

**Compute probability for a new document:**

$\vec{x} = $ "free money" = $\{1, 1\}$

$$
p(y=1|\vec{x})  =  \frac{p(y=1)\prod_j p(x_{ij}|y=1)}{p(\vec{x})} = \frac{p(y=1)p(free=1|y=1)p(money=1|y=1)}{p(\vec{x})}
$$

$$
= \frac{\frac{3}{7} \cdot \frac{2}{3} \cdot \frac{1}{3}}{p(\vec{x})}
$$

$$
p(y=-1|\vec{x}) = \frac{\frac{4}{7} \cdot \frac{2}{4} \cdot \frac{2}{4}}{p(\vec{x})}
$$

$$
p(\vec{x}) = (\frac{3}{7} \cdot \frac{2}{3} \cdot \frac{1}{3}) + (\frac{4}{7} \cdot \frac{2}{4} \cdot \frac{2}{4}) = 3
$$

thus

$$
p(y=1|\vec{x}) =  \frac{\frac{3}{7} \cdot \frac{2}{3} \cdot \frac{1}{3}}{3} \approx .476
$$

$$
p(y=-1|\vec{x}) =  \frac{\frac{4}{7} \cdot \frac{2}{4} \cdot \frac{2}{4}}{p(\vec{x})} \approx .524
$$

Note that $p(y=1|\vec{x}) + p(y=-1|\vec{x}) = 1$

**Laplacian smoothing: **

To avoid 0-probabilities in the calculations above, we can simply add a small value to each count, and likewise update the normalizer so terms sum to 1. For

$$p(x_{k}=1|y=1) = \frac{\sum_{(x_i, y_i) \in D}1[x_{ik}=1 \wedge y_i=1] + \epsilon}{2 \epsilon + \sum_{(x_i, y_i) \in D} 1[y_i=1]}$$

Commonly, $\epsilon=1$ is used (“plus one” smoothing).


**Multinomial Event Model**

The preceeding assumes a binary event model; that is, $x_{ij} \in \{0,1\}$. Alternatively, we can use term frequencies; i.e. $x_{ij} \in \mathcal{N}_+$. The term probabilities become:

$$
p(x_{ij}=1 | y_i = 1) = \frac{T_{1j}}{\sum_k T_{1k}}
$$

where $T_{cj}$ is the number occurrences of term $j$ in documents where $y=c$. E.g., count all the occurrences of the term $j$ in documents where the true class label is $c$. You can use the analogous equation for class $-1$, $p(x_{ij}=1 | y_i=-1)$.

Smoothing operates differently for Multinomial than Bernoulli Naive Bayes:

$$
p(x_{ij}=1 | y_i = 1) = \frac{T_{1j} + \epsilon}{|V|\epsilon + \sum_k T_{1k}}
$$

where $|V|$ is the number of unique terms in the vocabulary.

Note that in Multinomial Naive Bayes, to classify a new document, we only multiply terms that occur in the document: 

$$
p(y_i|x_i) = \frac{ p(y_i) \prod_{j \in n_i} p(x_{ij}|y_i)} {p(\vec{x})}
$$

where $n_i$ iterates over <span>**tokens**</span>, rather then <span>**terms**</span>. That is, if the term <span>*dog*</span> occurs twice in document $i$, its corresponding probability $p(dog|y_i)$ will appear twice in the product above.

See your book ([Ch13](http://nlp.stanford.edu/IR-book/pdf/13bayes.pdf)) for more details.


![mnnb](images/mnnb.png)